In [3]:
ls data/Negative/

^C


In [157]:
import random
import os
def group_image_by_prefix(image_list):
    image_dict = {}
    for image in image_list:
        prefix = "_".join(image.split("_")[:-1])
        if prefix not in image_dict:
            image_dict[prefix] = []
        image_dict[prefix].append(image)
    return image_dict

def subsample_images_positive(image_dict,max_samples_positive,output_path):
    sampled_images = []
    prefix_added = []
    number_of_file= 0
    os.system("mkdir -p "+output_path+"/Positive/")
    while number_of_file < max_samples_positive:
        new_list = [x for x in list(image_dict.keys()) if x not in prefix_added]
        prefix = random.choice(new_list)
        prefix_added.append(prefix)
        if len(image_dict[prefix]) > 0:
            sampled_images.append(image_dict[prefix])
            number_of_file +=len(image_dict[prefix])

    for images in sampled_images:
        for image in images:

            os.system("cp data/Positive/"+image+" "+output_path+"/Positive/"+image)


# Charger les fichiers
positive_files = os.listdir("data/Positive/")
number_of_positive_files = len(positive_files)
negative_files = os.listdir("data/Negative/")
number_of_negative_files = len(negative_files)
number_of_sampled_files = 10000
rate_of_positive_files = number_of_positive_files / (number_of_positive_files + number_of_negative_files)
max_samples_positive = int(number_of_sampled_files * rate_of_positive_files)
max_samples_negative = number_of_sampled_files - max_samples_positive
print(f"max_samples_positive: {max_samples_positive}")
print(f"max_samples_negative: {max_samples_negative}")
print(f"number_of_positive_files: {number_of_positive_files}")
print(f"number_of_negative_files: {number_of_negative_files}")
groups = group_image_by_prefix(positive_files)
subsample_images_positive(groups,max_samples_positive,f"data/sampled_{number_of_sampled_files}/")




max_samples_positive: 2810
max_samples_negative: 7190
number_of_positive_files: 111005
number_of_negative_files: 283921


In [156]:
def subsample_images_negative(image_dict,max_samples_negatiive,output_path):
    sampled_images = []
    prefix_added = []
    number_of_file= 0
    os.system("mkdir -p "+output_path+"/Negative/")
    while number_of_file < max_samples_negatiive:
        new_list = [x for x in list(image_dict.keys()) if x not in prefix_added]
        prefix = random.choice(new_list)
        prefix_added.append(prefix)
        if len(image_dict[prefix]) > 0:
            sampled_images.append(image_dict[prefix])
            number_of_file +=len(image_dict[prefix])

    for images in sampled_images:
        for image in images:

            os.system("cp data/Negative/"+image+" "+output_path+"/Negative/"+image)
groups = group_image_by_prefix(negative_files)

subsample_images_negative(groups,max_samples_negative,f"data/sampled_{number_of_sampled_files}/")
       

In [4]:
import os
import random
import shutil

# Chemins des dossiers
positive_dir = 'data/Positive'
negative_dir = 'data/Negative'
output_dir = 'data/sampled'

# Créer les dossiers de sortie
output_positive_dir = os.path.join(output_dir, 'Positive')
output_negative_dir = os.path.join(output_dir, 'Negative')
os.makedirs(output_positive_dir, exist_ok=True)
os.makedirs(output_negative_dir, exist_ok=True)


In [21]:
import os
import shutil

# Dossier contenant les fichiers originaux
input_dir = "data/Negative"
# Dossier où les sous-dossiers seront créés
output_dir = "data/Negative_grouped"    
os.makedirs(output_dir, exist_ok=True)

# Lister les fichiers dans le dossier
files = [f for f in os.listdir(input_dir) if os.path.isfile(os.path.join(input_dir, f))]

# Regrouper les fichiers par leur identifiant principal
grouped_files = {}
for file in files:
    # Extraire l'identifiant principal avant le dernier underscore '_'
    group_id = "_".join(file.split('_')[:-1])  # Tout sauf la dernière partie
    if group_id not in grouped_files:
        grouped_files[group_id] = []
    grouped_files[group_id].append(file)
i=0
# Créer des sous-dossiers et déplacer les fichiers
for group_id, group_files in grouped_files.items():
    group_folder = os.path.join(output_dir, group_id)
    os.makedirs(group_folder, exist_ok=True)
    for file in group_files:
        src_path = os.path.join(input_dir, file)
        dest_path = os.path.join(group_folder, file)
        shutil.move(src_path, dest_path)
    i+=1
    if i==10:
        break

print(f"Les fichiers ont été réorganisés dans {len(grouped_files)} sous-dossiers.")


Les fichiers ont été réorganisés dans 5555 sous-dossiers.


In [161]:
from PIL import Image
import re
import os

def reconstruct_image(files, input_dir, output_image_path):
    """
    Reconstruit une image à partir d'une liste de fichiers.

    :param files: Liste des fichiers contenant les sous-images.
    :param input_dir: Répertoire contenant les fichiers.
    :param output_image_path: Chemin pour sauvegarder l'image finale.
    """
    pattern = r"_(\d{1,8})x(\d{1,8})"
    x_max, y_max = 0, 0
    sub_images = {}
    c=0
    # Extraire les coordonnées et vérifier la validité des blocs
    for file in files:
        match = re.search(pattern, file)
        if match:
            y_start_y_end,x_start_x_end= map(str, match.groups())
            for i in range(1,len(x_start_x_end)):
                x_start = int(x_start_x_end[:i])
                x_end = int(x_start_x_end[i:])
                if x_end-x_start==256:
                    break
            for i in range(1,len(y_start_y_end)):
                y_start = int(y_start_y_end[:i])
                y_end = int(y_start_y_end[i:])
                if y_end-y_start==256:
                    break

            x_max = max(x_max, x_end)
            y_max = max(y_max, y_end)
            sub_images[file] = (x_start, y_start)
        else:
            print(f"Le fichier {file} ne correspond pas au format attendu.")    
    # Créer une image vide pour la reconstruction
    final_image = Image.new('RGB', (x_max, y_max))
    # Assembler l'image
    for file, (x_start, y_start) in sub_images.items():
        sub_image_path = os.path.join(input_dir, file)
        sub_image = Image.open(sub_image_path)
        final_image.paste(sub_image, (x_start, y_start))

    # Créer le répertoire cible si nécessaire
    os.makedirs(os.path.dirname(output_image_path), exist_ok=True)

    # Sauvegarder l'image finale
    final_image.save(output_image_path)
    print(f"Image reconstruite sauvegardée à : {output_image_path}")


# Chemins

input_dir = 'data/sampled_10000/Positive'
output_base_path = 'data/sampled_10000_regrouped/Positive'

# Lister les fichiers
all_files = os.listdir(input_dir)

# Grouper les fichiers par préfixe commun
pattern = r'(.+)_\w+\.jpeg'
prefixes = {re.match(pattern, f).group(1) for f in all_files if re.match(pattern, f)}
grouped_images = [[file for file in all_files if file.startswith(prefix)] for prefix in prefixes]
# Appliquer la fonction de reconstruction à chaque groupe
for group in grouped_images:
    group_name = re.match(pattern, group[0]).group(1)  # Utiliser le préfixe du groupe
    print(group)
    output_path = os.path.join(output_base_path, f"{group_name}_reconstructed.jpeg")
    reconstruct_image(group, input_dir, output_path)



['Lm_449818_20x_13_03_2019_b0s7c0x85843-2776y80606-2080m77_512768x512768.jpeg', 'Lm_449818_20x_13_03_2019_b0s7c0x85843-2776y80606-2080m77_7681024x0256.jpeg', 'Lm_449818_20x_13_03_2019_b0s7c0x85843-2776y80606-2080m77_256512x0256.jpeg', 'Lm_449818_20x_13_03_2019_b0s7c0x85843-2776y80606-2080m77_512768x0256.jpeg', 'Lm_449818_20x_13_03_2019_b0s7c0x85843-2776y80606-2080m77_7681024x256512.jpeg', 'Lm_449818_20x_13_03_2019_b0s7c0x85843-2776y80606-2080m77_512768x256512.jpeg', 'Lm_449818_20x_13_03_2019_b0s7c0x85843-2776y80606-2080m77_0256x0256.jpeg', 'Lm_449818_20x_13_03_2019_b0s7c0x85843-2776y80606-2080m77_512768x10241280.jpeg', 'Lm_449818_20x_13_03_2019_b0s7c0x85843-2776y80606-2080m77_256512x512768.jpeg', 'Lm_449818_20x_13_03_2019_b0s7c0x85843-2776y80606-2080m77_10241280x256512.jpeg', 'Lm_449818_20x_13_03_2019_b0s7c0x85843-2776y80606-2080m77_0256x512768.jpeg', 'Lm_449818_20x_13_03_2019_b0s7c0x85843-2776y80606-2080m77_0256x256512.jpeg', 'Lm_449818_20x_13_03_2019_b0s7c0x85843-2776y80606-2080m77_2